# Downloading the sitemap

In [5]:
import requests

In [6]:
SITEMAP_URL = 'https://www.meilleurtaux.com/sitemap/xml.html'
# TODO: verifier le robot.txt

In [7]:
response = requests.get(SITEMAP_URL) # TODO: Ajouter headers et timeout
response.raise_for_status()


In [8]:
xml_content = response.content
len(xml_content)

4010754

# Parsing the xml

In [9]:
import xml.etree.ElementTree as ET

In [10]:
root = ET.fromstring(xml_content)
root

<Element '{http://www.sitemaps.org/schemas/sitemap/0.9}urlset' at 0x11a515b70>

In [11]:
SITEMAP_NAMESPACE = {'sm': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

In [12]:
url_elements = root.findall('sm:url', SITEMAP_NAMESPACE)
url_elements[:3]

[<Element '{http://www.sitemaps.org/schemas/sitemap/0.9}url' at 0x11a515c10>,
 <Element '{http://www.sitemaps.org/schemas/sitemap/0.9}url' at 0x11a515e40>,
 <Element '{http://www.sitemaps.org/schemas/sitemap/0.9}url' at 0x11a516020>]

In [13]:
url_elements[10].find('sm:loc', SITEMAP_NAMESPACE).text

'https://www.meilleurtaux.com/credit-immobilier/simulation-de-pret-immobilier/prix-ventes-immo.html'

In [14]:
urls = [
url_elem.find('sm:loc', SITEMAP_NAMESPACE).text
for url_elem in url_elements
]

In [15]:
len(urls)

15604

# subset and cleaning urls

In [16]:
urls[:5]

['https://www.meilleurtaux.com/financement-scpi/',
 'https://www.meilleurtaux.com/credit-immobilier/notre-analyse-des-taux.html',
 'https://www.meilleurtaux.com/credit-immobilier/notre-analyse-des-taux/evolution-oat-tec-10-ans.html',
 'https://www.meilleurtaux.com/credit-immobilier/notre-analyse-des-taux/evolution-taux-directeur-bce.html',
 'https://www.meilleurtaux.com/credit-immobilier/notre-analyse-des-taux/taux-immobilier-15-ans.html']

In [17]:
urls_credit_immo = [url for url in urls if 'https://www.meilleurtaux.com/credit-immobilier/' in url]
len(urls_credit_immo) # la, si on veut pas y passer la nuit, il va falloir paralleliser vu le nombre d'url

5107

In [18]:
any(['?' in c for c in urls_credit_immo]) # no params :)

False

 pour contourner les 403 sans utiliser de proxy 

In [19]:
import random
import logging # Ajouté pour info si besoin

USER_AGENTS = [

    # --- Chrome ---
    # Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', # Win 11
    # macOS
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_7_10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', # macOS 11 Big Sur
    # Linux
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:115.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', # Chrome on Ubuntu

    # --- Firefox ---
    # Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0',
    'Mozilla/5.0 (Windows NT 11.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0', # Win 11
    # macOS
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0',
    # Linux
    'Mozilla/5.0 (X11; Linux x86_64; rv:123.0) Gecko/20100101 Firefox/123.0',
    'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:122.0) Gecko/20100101 Firefox/122.0', # Firefox 32bit on Ubuntu

    # --- Edge ---
    # Windows
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.65',
    'Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.2365.80', # Win 11
    # macOS
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.65',

    # --- Safari ---
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3.1 Safari/605.1.15', # macOS Monterey/Ventura/Sonoma often have similar Safari versions
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_6_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3.1 Safari/605.1.15', # Ventura
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_3_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3.1 Safari/605.1.15', # Sonoma

]

def get_random_headers():
    """Génère un dictionnaire de headers HTTP réalistes et variés."""

    # 1. Choisir un User-Agent au hasard
    user_agent = random.choice(USER_AGENTS)

    # 2. Initialiser les headers de base
    headers = {
        'User-Agent': user_agent,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
    }

    # 3. Ajouter des headers "Sec-" (sécurité/contexte de la requête) - courants dans les navigateurs modernes
    headers['Sec-Fetch-Dest'] = 'document'
    headers['Sec-Fetch-Mode'] = 'navigate'
    headers['Sec-Fetch-Site'] = 'none'
    headers['Sec-Fetch-User'] = '?1'

    # 4. Ajouter des headers "Sec-CH-UA" (Client Hints - spécifiques aux navigateurs basés sur Chromium comme Chrome, Edge)
    if 'Chrome' in user_agent or 'Edg' in user_agent:
        # Extraire la version majeure (simpliste, peut échouer sur certains formats)
        major_version = "123" # Version par défaut si extraction échoue
        try:
            if 'Chrome/' in user_agent and 'Edg/' not in user_agent:
                major_version = user_agent.split('Chrome/')[1].split('.')[0]
            elif 'Edg/' in user_agent:
                 major_version = user_agent.split('Edg/')[1].split('.')[0]
        except:
             pass # Garder la version par défaut

        # Construire un header Sec-CH-UA plausible
        if 'Edg/' in user_agent: # Cas Edge
             headers['Sec-CH-UA'] = f'"Microsoft Edge";v="{major_version}", "Not(A:Brand";v="99", "Chromium";v="{major_version}"'
        else: # Cas Chrome (ou autre basé sur Chromium sans être Edge)
             headers['Sec-CH-UA'] = f'"Google Chrome";v="{major_version}", "Not(A:Brand";v="99", "Chromium";v="{major_version}"'

        # Ajouter Sec-CH-UA-Mobile basé sur la présence de "Mobile" dans l'UA
        headers['Sec-CH-UA-Mobile'] = '?1' if 'Mobile' in user_agent else '?0'

        # Ajouter Sec-CH-UA-Platform basé sur l'OS détecté dans l'UA (simplifié)
        if 'Windows' in user_agent:
            headers['Sec-CH-UA-Platform'] = '"Windows"'
        elif 'Macintosh' in user_agent or 'Mac OS X' in user_agent:
             headers['Sec-CH-UA-Platform'] = '"macOS"'

    return headers


print("Exemple de headers générés aléatoirement :\n")
random_headers_example = get_random_headers()
for key, value in random_headers_example.items():
    print(f"{key}: {value}")

    # Utilisation dans une requête (simulée) :
    # try:
    #     url_to_check = "https://www.meilleurtaux.com/credit-immobilier/" # Exemple d'URL
    #     request_headers = get_random_headers()
    #     print(f"\nHeaders envoyés pour {url_to_check} :\n{request_headers}")
    #     # response = requests.head(url_to_check, headers=request_headers, timeout=15, allow_redirects=True)
    #     # print(f"\nStatus Code reçu : {response.status_code}")
    #     # print(f"URL Finale : {response.url}")
    # except Exception as e:
    #     print(f"\nErreur lors de la requête : {e}")

Exemple de headers générés aléatoirement :

User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:122.0) Gecko/20100101 Firefox/122.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7
Accept-Language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
DNT: 1
Upgrade-Insecure-Requests: 1
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: none
Sec-Fetch-User: ?1


In [ ]:
import concurrent.futures 
import time
import random

MAX_WORKERS = 5 
DELAY_PER_REQUEST = 0.5

def check_url(url):

    random_delay = random.uniform(DELAY_PER_REQUEST, DELAY_PER_REQUEST + 0.5)
    time.sleep(random_delay)
    request_headers = get_random_headers()
    response = requests.head(url, headers=request_headers, timeout=15, allow_redirects=True)
    #response = requests.head(url, timeout=10, allow_redirects=True)
    return {
        'status': response.status_code,
        'original_url': url,
        'final_url': response.url # au cas ou il y aurait des redicrection
    }

urls_valides = []

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:

    future_to_url = {executor.submit(check_url, url): url for url in urls_credit_immo}

    processed_count = 0
    total_urls = len(urls_credit_immo)

    for future in concurrent.futures.as_completed(future_to_url):
        url_result = future.result()
        processed_count += 1
        status = url_result['status']
        original_url = url_result['original_url']
        final_url = url_result['final_url']

        if status == 200:
            if original_url != final_url: # au cas ou il y ai de redirection
                 print(f"URL {original_url} redirect vers {final_url}")
            urls_valides.append(final_url)
        else:
            print(f"URL {original_url} PROBLEM !! ({status})")

        if processed_count % 100 == 0 or processed_count == total_urls:
             print(f"Progression : {processed_count}/{total_urls} URLs vérifiées...")

print(len(urls_valides), 'urls validés')

Progression : 100/5107 URLs vérifiées...
